In [1]:
import pandas as pd 
import pyarrow as pa 
import pyarrow.parquet as pq 
import warnings
warnings.filterwarnings('ignore')

In [4]:
#caragamos el dataset de steam_games
df_steamgames = pd.read_parquet('dataset_limpio/steam_games.parquet')
df_steamgames.head(2)

,publisher,app_name,reviews_url,price,item_id,developer,release_year,genres
0,Kotoshiro,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,Kotoshiro,2018,Action
1,"Making Fun, Inc.",Ironbound,http://steamcommunity.com/app/643980/reviews/?...,0.00,643980,Secret Level SRL,2018,Free to Play


In [5]:
#cargamos el dataset de users_items
df_users_items = pd.read_parquet('dataset_limpio/items_normalized.parquet')
df_users_items.head(2)

,item_id,item_name,steam_id,items_count,user_id,playtime_hours
0,10,Counter-Strike,76561197970982479,277,76561197970982479,0.10
1,30,Day of Defeat,76561197970982479,277,76561197970982479,0.12


In [19]:
#cargamos el dataset de reviews
df_steamgames_and_reviews = pd.read_parquet('dataset_funciones/steamgames_and_reviews.parquet')
df_steamgames_and_reviews.head()


,publisher,app_name,reviews_url,price,item_id,developer,release_year,genres,user_id,recommend,posted_year,sentiment_analysis
0,Octoshark Studios,"Pirates, Vikings, and Knights II",http://steamcommunity.com/app/17570/reviews/?b...,0.0,17570,Octoshark Studios,2010,Action,76561198010653835,True,2015,2
1,Octoshark Studios,"Pirates, Vikings, and Knights II",http://steamcommunity.com/app/17570/reviews/?b...,0.0,17570,Octoshark Studios,2010,Action,76561198075581042,True,2014,2
2,Octoshark Studios,"Pirates, Vikings, and Knights II",http://steamcommunity.com/app/17570/reviews/?b...,0.0,17570,Octoshark Studios,2010,Action,GameBrak,True,2015,2
3,Octoshark Studios,"Pirates, Vikings, and Knights II",http://steamcommunity.com/app/17570/reviews/?b...,0.0,17570,Octoshark Studios,2010,Action,CjDoesIntros,True,2014,1
4,Octoshark Studios,"Pirates, Vikings, and Knights II",http://steamcommunity.com/app/17570/reviews/?b...,0.0,17570,Octoshark Studios,2010,Action,76561198075920681,True,2015,1


### Función `PlayTimeGenre`

Esta función devuelve el año con mas horas de juego para un género

In [6]:
def PlayTimeGenre(genero, df_steamgames: pd.DataFrame, df_users_items: pd.DataFrame):
    # Convertimos el género proporcionado a minúsculas para una comparación sin distinción de mayúsculas
    genero = genero.lower()
    
    # Filtramos los juegos por género de manera más flexible
    df_filtered = df_steamgames[df_steamgames['genres'].str.lower().str.contains(fr'\b{genero}\b', na=False)]

    if not df_filtered.empty:
        # Convertimos 'item_id' a tipo de dato object en ambos DataFrames
        df_users_items['item_id'] = df_users_items['item_id'].astype(str)
        df_filtered['item_id'] = df_filtered['item_id'].astype(str)

        df_merged = pd.merge(df_users_items, df_filtered[['item_id', 'release_year']], left_on='item_id', right_on='item_id')
        
        # Verificamos si la longitud de df_filtered es mayor a cero, es decir, se encontraron géneros
        result = {"Año con más horas jugadas para el género {}: {}".format(genero.capitalize(), df_merged.groupby('release_year')['playtime_hours'].sum().idxmax())}
    else:
        result = {"Año con más horas jugadas para género {}: {}".format(genero.capitalize(), "Género no encontrado en la base de datos")}

    return result

In [9]:
PlayTimeGenre('Action', df_steamgames, df_users_items)

{'Año con más horas jugadas para el género Action: 2012'}

### Función `UserForGenre`

Función que devuelve el usuario con mas cantidad de horas jugadas para un genero

In [10]:
def UserForGenre(genero: str, df_steamgames: pd.DataFrame, df_users_items: pd.DataFrame):
    # Convertimos el género proporcionado a minúsculas para una comparación sin distinción de mayúsculas
    genero = genero.lower()
    
    # Filtramos los juegos por género de manera más flexible
    df_filtered = df_steamgames[df_steamgames['genres'].str.lower().str.contains(fr'\b{genero}\b', na=False)]
    
    if not df_filtered.empty:
        df_users_items['item_id'] = df_users_items['item_id'].astype(str)
        
        # Convertimos la columna 'item_id' en df_filtered a tipo 'str'
        df_filtered['item_id'] = df_filtered['item_id'].astype(str)
        
        df_merged = pd.merge(df_users_items, df_filtered[['item_id', 'release_year']], on='item_id')
        
        # Filtramos los datos desde el año 2003 en adelante
        df_merged = df_merged[df_merged['release_year'] >= 2003]
        user_with_most_playtime = df_merged.groupby('user_id')['playtime_hours'].sum().idxmax()
        playtime_by_year = df_merged.groupby(['release_year', 'user_id'])['playtime_hours'].sum().reset_index()
        playtime_by_year = playtime_by_year[playtime_by_year['user_id'] == user_with_most_playtime]
        playtime_by_year = playtime_by_year.rename(columns={'release_year': 'Año', 'playtime_hours': 'Horas'})
        
        # Convertimos las horas a enteros
        playtime_by_year['Horas'] = playtime_by_year['Horas'].astype(int)
        
        result = {
            "Usuario con más horas jugadas para Género {}:".format(genero.capitalize()): user_with_most_playtime,
            "Horas jugadas": [{"Año": str(row['Año']), "Horas": row['Horas']} for _, row in playtime_by_year.iterrows()]
        }
    else:
        result = {"Usuario con más horas jugadas para Género {}:".format(genero.capitalize()): "Género no encontrado en la base de datos", "Horas jugadas": []}
    
    return result

In [12]:
UserForGenre('Indie', df_steamgames, df_users_items)

{'Usuario con más horas jugadas para Género Indie:': 'wolop',
 'Horas jugadas': [{'Año': '2006', 'Horas': 10712}]}

### Función `UsersRecommend`

Función que devuelve el Top 3 de los juegos mas recomendados por año

In [20]:
def UsersRecommend(año: int, df_steamgames_and_reviews: pd.DataFrame):
    # Verificamos si el año está dentro del rango esperado
    rango_aceptado = range(2010, 2016)
    if año not in rango_aceptado:
        return {"message": "Solo existen registros entre 2010 y 2015"}

    # Filtramos por el año deseado
    df_filtered = df_steamgames_and_reviews[df_steamgames_and_reviews['release_year'] == año]

    # Filtramos por comentarios recomendados y sentiment_analysis positivo/neutral
    df_filtered = df_filtered[(df_filtered['recommend'] == True) & (df_filtered['sentiment_analysis'].isin([0, 1, 2]))]

    # Obtenemos el top 3 de juegos recomendados
    top_games = df_filtered['app_name'].value_counts().head(3).reset_index()
    top_games = top_games.rename(columns={'index': 'Puesto 1', 'app_name': 'Juego'})

    # Modificamos la estructura del resultado
    result = [{"Puesto {}".format(i + 1): juego, 'recomendaciones': count} for i, (juego, count) in enumerate(zip(top_games['Juego'], top_games['count']))]

    return result

In [23]:
UsersRecommend(2015, df_steamgames_and_reviews)

[{'Puesto 1': 'Grand Theft Auto V', 'recomendaciones': 272},
 {'Puesto 2': 'Rocket League®', 'recomendaciones': 260},
 {'Puesto 3': 'Undertale', 'recomendaciones': 215}]

### Función `UsersWorstDeveloper`

Función que devuelve el Top 3 de los peores desarrolladores del año

In [25]:
def UsersWorstDeveloper(año: int, df_steamgames_and_reviews: pd.DataFrame):
    # Verificamos si el año está dentro del rango esperado
    rango_aceptado = range(2010, 2016)
    if año not in rango_aceptado:
        return {"message": "solo existen registros entre 2010 y 2015"}

    # Filtramos por comentarios no recomendados y sentiment_analysis negativo
    df_filtered = df_steamgames_and_reviews[(df_steamgames_and_reviews['recommend'] == False) & (df_steamgames_and_reviews['sentiment_analysis'] == 0)]

    # Filtramos por el año deseado
    df_filtered_year = df_filtered[df_filtered['release_year'] == año]

    # Si no hay datos para el año, retornamos mensaje
    if not df_filtered_year.empty:
        # Obtener los top 3 desarrolladores con menos recomendaciones
        top_developers = df_filtered_year['developer'].value_counts().head(3).reset_index()
        top_developers = top_developers.rename(columns={'index': 'Puesto 1', 'developer': 'Desarrollador'})

        # Modificamos la estructura del resultado
        result = [{"Puesto {}".format(i + 1): desarrollador} for i, desarrollador in enumerate(top_developers['Desarrollador'])]
    else:
        result = {"No hay juegos no recomendados para el año {}".format(año)}

    return result

In [27]:
UsersWorstDeveloper(2011, df_steamgames_and_reviews)

[{'Puesto 1': 'Reloaded Productions'},
 {'Puesto 2': 'Trashmasters'},
 {'Puesto 3': 'Re-Logic'}]

### Función `sentiment_analysis`

Función que devuelve el conteo de las reviews positivas, neutrales y negativas de un desarrollador

In [28]:
def sentiment_analysis(empresa_desarrolladora: str, df_steamgames_and_reviews: pd.DataFrame):
    # Filtramos por desarrolladora
    df_filtered_developer = df_steamgames_and_reviews[df_steamgames_and_reviews['developer'] == empresa_desarrolladora]

    # Verificamos que haya datos para la desarrolladora
    if not df_filtered_developer.empty:
        # Contamos los sentimientos y mapeamos el número del sentimiento a su etiqueta correspondiente
        sentiment_counts = df_filtered_developer['sentiment_analysis'].value_counts().reset_index(drop=True)
        sentiment_mapping = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
        sentiment_counts.index = sentiment_counts.index.map(sentiment_mapping)
        
        result = {empresa_desarrolladora: sentiment_counts.to_dict()}
    else:
        result = 'No cuento con los registros de esa empresa en mi base de datos'

    return result

In [34]:
sentiment_analysis('Ubisoft', df_steamgames_and_reviews)

{'Ubisoft': {'Negative': 52, 'Neutral': 32, 'Positive': 19}}